In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from zipfile import ZipFile
import zipfile_deflate64
import os
from pathlib import Path
import sys
import glob
from os.path import join
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio 
from rasterio.warp import calculate_default_transform, reproject, Resampling
import rasterio.mask
from pyproj import CRS
import matplotlib.pyplot as plt

# Configure files and other info

In [17]:
# It could make sense to have a lib/ style directory
# like PLACES has for common functionality
# and this code block would be useful there for getting
# a fr() path

# Filepath directories

# Get the absolute path to the project directory
# Which is one directory above notebooks/
ABS_DIR = os.path.abspath(Path(os.getcwd()).parents[0])
# Get raw data directory
FR = join(ABS_DIR, 'data', 'raw')
# Get interim data directory
FI = join(ABS_DIR, 'data', 'interim')
# Get processed data directory
FP = join(ABS_DIR, 'data', 'processed')

# Directories for raw exposure, vulnerability (vuln) and 
# administrative reference files
#  all exist so just need references
EXP_DIR_R = join(FR, 'exposure')
VULN_DIR_R = join(FR, 'vuln')
REF_DIR_R = join(FR, 'ref')
# Haz is for FEMA NFHL and depth grids
HAZ_DIR_R = join(FR, 'haz')

# Directories for interim exposure, vulnerability (vuln) and 
# hazard
EXP_DIR_I = join(FI, 'exposure')
VULN_DIR_I = join(FI, 'vuln')
HAZ_DIR_I = join(FI, 'haz')

# Ensure they exist
Path(EXP_DIR_I).mkdir(parents=True, exist_ok=True)
Path(VULN_DIR_I).mkdir(parents=True, exist_ok=True)
Path(HAZ_DIR_I).mkdir(parents=True, exist_ok=True)

# Reference fips
FIPS = '42101'

# Unzip and move files to interim

In [4]:
# For each .zip directory in fr
# Create needed subdirectories in interim/
# Unzip in the appropriate interim/ subdirectory

for path in Path(FR).rglob('*.zip'):
    # Avoid hidden files and files in directories
    if path.name[0] != '.':
        # Get root for the directory this .zip file is in
        zip_root = path.relative_to(FR).parents[0]

        # Get path to interim/zip_root
        zip_to_path = join(FI, zip_root)

        # Make directory, including parents
        # No need to check if directory exists bc
        # it is only created when this script is run
        Path(zip_to_path).mkdir(parents=True, exist_ok=True)

        # Unzip to zip_to_path
        with ZipFile(path, 'r') as zip_ref:
            zip_ref.extractall(zip_to_path)


# Obtain base data for structure inventories

In [4]:
# Parcels are the core exposure element 
# of building-based flood risk assessments
# This can be represented by appraisal or NSI data

# First, we want to clean and filter the parcel dataset for
# the information we need for our flood risk assessment
# Then, we want to link parcels with all of the other information
# that we will use in our flood risk assessment
# This includes attribute links and spatial links
# for things like footprints, flood hazard, administrative references

# We want to link parcels to other spatial features in the
# CRS of those spatial features. This will limit 
# expensive spatial processing for tasks like reprojection. For example,
# if we were to reproject every flood hazard depth grid to the
# parcel CRS, we would need to perform many redundant reprojection tasks.
# Instead, we could reproject the parcels one time and overlay this with
# all the depth grids. A downside of this approach is that if we want to
# plot depth grids and parcels, we would likely want to do this in 
# the parcel CRS. Since plotting is an occassional task, but linking
# parcels & depths is a ubiquitous one, I think it's better to do the
# parcel reprojection for spatial merges. There are also layers that will
# share the CRS of the parcels since WGS84 is very common for parcel
# datasets, reference files like block groups, etc. 

## Process NSI

In [7]:
# The NSI comes with all the data necessary for performing a standard 
# flood risk assessment. It is still useful to process the raw data.
# Here, we subset to residential properties with 1 to 2 stories
# and save as a geodataframe. These are the types of residences we have
# multiple depth-damage functions for and a literature base to draw 
# from to introduce uncertainty in these loss estimates

# We are going to use the following occupancy types 
# RES1 - single family residences
# Res2 - manufactured home
# RES 3 - multifamily residences
# Only if they have <= 2 stories (we lack info for specing 
# the kinds of DDFs we'd like to -- w/ uncertainty -- for 3 story houses)
# RES2 
# While DDFs are not specific to manufactured homes, as long
# as we know basement type, foundation height, and
# stories, we can use the requisite DDF archetype

# Future versions of this framework will not subset to the residential
# properties as done here, but since we don't have the 
# requisite depth-damage function information for other properties,
# it would be a waste of storage space and processing time to 
# save/process a larger NSI .gpkg file at this time

In [16]:
# Read raw NSI data
nsi_filep = join(EXP_DIR_R, 'nsi.pqt')
# Read and reset index
nsi_full = pd.read_parquet(nsi_filep).reset_index(drop=True)

# Convert to geodataframe
geometry = gpd.points_from_xy(nsi_full['properties.x'],
                             nsi_full['properties.y'])
# The NSI CRS is EPSG 4326
nsi_gdf = gpd.GeoDataFrame(nsi_full, geometry=geometry,
                           crs="EPSG:4326")

# Drop the following columns
drop_cols = ['type', 'geometry.type', 'geometry.coordinates']
nsi_gdf = nsi_gdf.drop(columns=drop_cols)

# Remove "properties" from columns
col_updates = [x.replace("properties.", "") for x in nsi_gdf.columns]
nsi_gdf.columns = col_updates

In [35]:
# Subset to residential properties and update
# occtype category for easier use in loss estimation steps

# Get residential structures
nsi_res = nsi_gdf.loc[nsi_gdf['st_damcat'] == 'RES']


# Add #story and wb (with basement) or nb (no basement) to RES3 homes
# Store NB or WB indexed to RES3 homes based on B,C and N found_type
# Get num_story + 'S' 
# Merge these and then add to occtype for RES3 homes

# Start with index of res3 homes
res3_ind = nsi_res['occtype'].str[:4] == 'RES3'
# Get subsetted df
res3 = nsi_res.loc[res3_ind]

# For this subset
# If found_type == B, then WB
# Else then NB
res3b = np.where(res3['found_type'] == 'B',
                 'WB',
                 'NB')
# For this subset
# Get num_story + 'S'
res3s = res3['num_story'].astype(str) + 'S'

# Adjust occtype column for these homes in nsi_res
nsi_res.loc[res3_ind, 'occtype'] = res3['occtype'] + '-' + res3s + res3b

# For this case-study, don't use any building with more 
# than 2 stories
res1_3s_ind = nsi_res['num_story'] > 2
# Final residential dataframe
res_f = nsi_res.loc[~res1_3s_ind]

In [56]:
# Subset to relevant columns
cols = ['fd_id', 'occtype', 'found_type', 'cbfips',
        'ftprntsrc', 'found_ht', 'val_struct',
        'val_cont', 'source', 'firmzone', 'ground_elv_m',
        'geometry']

res_out = res_f.loc[:,cols]

# Write out to interim/exposure/
EXP_OUT_FILEP = join(EXP_DIR_I, 'nsi_res.gpkg')
nsi_res.to_file(EXP_OUT_FILEP, driver='GPKG')

## Process appraiser data

In [66]:
# Load philadelphia water department parcels
pc_pwd = gpd.read_file(join(EXP_DIR_R, 'pc_pwd.gpkg'))

# Load philadelphia office of property assessment parcels
pc_opa = pd.read_csv(join(EXP_DIR_R, 'pc_opa.csv'))

# Load building footprint data
bld_fp = gpd.read_file(join(EXP_DIR_R, 'bld_fp.gpkg'))

/tmp/ipykernel_974319/1962128361.py:5: DtypeWarning: Columns (12,21,25,30,54,67,69,76) have mixed types. Specify dtype option on import or set low_memory=False.
  pc_opa = pd.read_csv(join(EXP_DIR_R, 'pc_opa.csv'))


In [67]:
# Remove "properties." from the pc_pwd column names
col_updates = [x.replace("properties.", "") for x in pc_pwd.columns]
pc_pwd.columns = col_updates

# Retain relevant columns
col_keep = ['ADDRESS', 'PARCELID', 'ADDRESS',
            'PIN', 'BRT_ID', 'geometry']
pc_pwd = pc_pwd.loc[:, col_keep]

In [68]:
# Remove "properties." from the bld_fp column names
col_updates = [x.replace("properties.", "") for x in bld_fp.columns]
bld_fp.columns = col_updates

# Retain relevant columns
col_keep = ['BIN', 'ADDRESS', 'BASE_ELEVATION',
            'PARCEL_ID_NUM', 'Shape__Area', 'geometry']
bld_fp = bld_fp.loc[:, col_keep]

In [69]:
# Filter pc_opa and retain relevant columns
# Reference https://metadata.phila.gov/#home/datasetdetails/
# 5543865f20583086178c4ee5/representationdetails/55d624fdad35c7e854cb21a4/

# assessment_date: important for understanding the potential
# for renovations/improvements/degradation

# basements: detailed fields can be aggregated for whether a building
# has a basement or not. whether it is finished or not does not matter
# for flood risk estimation other than the effect it may have on 
# structure value - this is an "observed" field with a different column

# category_code: identify residential properties. subset to these
# 1 - single family, 2 - multi family,
# 3 - mixed used, 14 - apartments > 4 units

# market_value: certified market value (includes land and buildings)

# market_value_date: important for getting market value in 
# correct real value, but it is fully null. so we will use
# assessment_date as a proxy for this. 

# number_stories: # stories above ground level

# parcel_number: matches brt_id in PWD parcels

# year_built: useful for determining which building codes
# construction was subject to

# year_built_estimate: indicates if the year_built column is an estimate

# pin: use to link to PIN in PWD data

In [84]:
# How many observations do we start with
print('Number OPA Properties: ' + str(len(pc_opa)))

# Filter pc_opa to category codes 1, 2, 3, and 14
cat_codes = [1, 2, 3, 14]
pc_opa_f = pc_opa.loc[pc_opa['category_code'].isin(cat_codes)]

print('Retained Properties in category codes 1, 2, 3, and 14: ' 
      + str(len(pc_opa_f)))

# Filter to number stories 1 or 2
# FLAG: should we take the assessor data at face value?
pc_opa_f = pc_opa_f.loc[(pc_opa_f['number_stories'] == 1) |
                        (pc_opa_f['number_stories'] == 2)]

print('Retained Properties with 1 or 2 Stories: ' 
      + str(len(pc_opa_f)))

# Keep properties with non-zero and non-null market values
pc_opa_f = pc_opa_f.loc[(pc_opa_f['market_value'].notnull())
                        & (pc_opa_f['market_value'] != 0)]

# Retain relevant columns
keep_cols = ['assessment_date', 'category_code',
             'market_value', 'number_stories',
             'basements',
             'year_built', 'year_built_estimate',
             'parcel_number', 'pin',]
pc_opa_f = pc_opa_f.loc[:, keep_cols]

# How many observations do we filter to
print('Retained Properties with non-zero or non-null market values: ' 
      + str(len(pc_opa_f)))

Number OPA Properties: 582341
Number OPA Properties in category codes 1, 2, 3, and 14: 521059
Number OPA Properties with 1 or 2 Stories: 437855
Number OPA Properties with non-zero or non-null market values: 437844


In [92]:
# Retain properties with an assessment date 
pc_opa_f = pc_opa_f[pc_opa_f['assessment_date'].notnull()]

# Get year of assessment date
# Have to split the string to use pd.to_datetime successfully
dates = pd.to_datetime(pc_opa_f['assessment_date'].str.split(' ').str[0],
                       format="%Y-%m-%d")
years = dates.dt.year

# Use the column name "Year" to match with the hpi data
# for better syntax on merge
pc_opa_f = pc_opa_f.assign(Year = years)

# Deflate market values in terms of 2022 value
# For assessments in 2022, no adjustment needed

# Read hpi deflator data
hpi_path = join(EXP_DIR_R, 'hpi_county.xlsx')
# Manual inspection of file provides the values 
# for skiprows, HPI as float, FIPS as str
hpi = pd.read_excel(hpi_path, skiprows=6, dtype={'FIPS code': 'str'})

# Subset to our county
hpi_fips = hpi.loc[hpi['FIPS code'] == FIPS]

# I can't do dtype 'HPI': 'float' but we need to convert the column
# and the below works fine
hpi_fips = hpi_fips.assign(hpi=hpi_fips['HPI'].astype(float))

# Get HPI with 2022 base
# TODO: Define in cfg file
YR_BASE = 2022

# Do this by dividing HPI values by the HPI value 
# for the row with Year = 2022
# Round to hundredth place
# TODO: better column name than hpi_2022 (hpi_base?)
base_hpi = hpi_fips.loc[hpi_fips['Year'] == YR_BASE, 'hpi'].values[0]
rounded_hpi = round(hpi_fips.loc[:, 'hpi']/base_hpi, 2)
hpi_fips = hpi_fips.assign(hpi_2022=round(hpi_fips['hpi']/base_hpi, 2))

# Merge the ratio with the parcel dataframe
# TODO: it could make sense to add FIPS as a merge
# when expanding to more locations
# I like changing df names after merges because
# if you need to rerun a code block it will work without
# column name or index issues
pc_opa_f2 = pc_opa_f.merge(hpi_fips[['Year', 'hpi_2022']],
                           on='Year',
                           how='left')

# Making the assumption that assessments in 2023 have 
# similar value to those from 2022, otherwise we get null entries
pc_opa_f2.loc[pc_opa_f2['Year'] == 2023, 'hpi_2022'] = 1

# Scale the market value by 1/ratio 
pc_opa_f2 = pc_opa_f2.assign(bld_mv_2022=round(.8*pc_opa_f2['market_value']
                             /pc_opa_f2['hpi_2022']))

In [127]:
# Map basements column to basement types
# 0. None – Indicates no basement.
# A. Full Finished – Occupies the entire area under the first floor.
# B. Full Semi-Finished – Could have some finish to include a floor covering,
# and ceiling. It looks more like a living area rather than a basement.
# C. Full Unfinished – Is a typical basement with unfinished concrete floor,
# either rubble stone or cement over stone or concrete walls and would have
# exposed wood joist ceilings.
# D. Full – Unknown Finish
# E. Partial Finished – Occupies a portion under the first floor. Be careful of
# areas under sheds and porches. If there is a garage at basement level then it is a
# partial basement.
# F. Partial Semi-Finished – One or more finished areas.
# G. Partial Unfinished
# H. Partial - Unknown Finish
# I. Unknown Size - Finished
# J. Unknown Size - Unfinished

# From my property.phila.gov, google maps, opa data groundtruth checks,
# each example of null in opa data matches no basement in property.phila.gov
# and seems visually plausible. So, fill na with 0 which corresponds to
# no basement
# The categories w/ letters should be marked as basements
# There is no way to know which of the no basement homes are crawl
# space homes versus slab or other type of foundation
# Further, it's unclear if some crawl space homes may be considered
# partial basements. It doesn't appear like this is the case, but
# it's uncertain. The way I'm going to do this, na cells become no basement
# and we will split no basement into slab & crawl space foundation types
# based on NFIP Poliices foundation type proportions
# I have no information on pilings/piers and other raised foundations
# so will assume they are not in Philadelphia. This could be wrong but 
# I don't have other information about this

# There are also categories 1, 2, 3, and 4 which have no description
# in the metadata. So will treat these as no basement
# Searching on property.phila.gov, there is nothing said about
# the basement type for the properties I've checked

# basement or not columns
pc_opa_f2['b_type'] = 'NB'
# if basements not 0 or null, 'WB'
pc_opa_f2.loc[(pc_opa_f2['basements'].notnull()) &
              ~(pc_opa_f2['basements'].isin(['0', '1', '2', '3', '4'])),
              'b_type'] = 'WB'

In [ ]:
# Drop any redundant columns (i.e. don't need Year anymore)

In [ ]:
# Merge pwd with bld_fp
# Keep the bld_fp polygons

# Merge resultant gdf by PIN/pin with opa
# Check how results compare to merging on brt and parcel_number

# Only keep properties with a match. Remainder are ambiguous
# Instructed by OPA that PWD and condo matching will be mostly
# unsuccessful with these methods. Condos may need more attention later
# Discrepancies in loss estimates may be due to ambiguous processing 
# decisions about which buildings to include and how to estimate losses
# for them. An important point may be that reporting absolute losses
# is so sensitive to an uncertain extensive margin that it should not
# be done without explicitly reporting how many properties are ambiguous
# property types

In [ ]:
# Evaluate results of merge and determine which building_code_description
# parcels to keep

In [ ]:
# Write out building footprint centroids for parcel ids
# Some parcels have multiple buildings on them (these are tax parcels)
# and we want to have unique ids for these

# Write out the unique ids as .pqt with relevant columns for loss estimation

In [ ]:
# Match pc_pwd and pc_opa on PIN
# The PARCELID column in the resulting dataset
# can be used to link to PARCEL_ID_ in bld_fp
# This avoids relying on potentially inaccurate spatial joins
# since we only have point data for opa parcels and 
# footprint data for buildings
# TODO: The more general processing step is for
# parcel polygon and building footprint joins. Not all 
# future sources of appraiser data will provide the links
# like the Philadelphia data from PWD. For best performance in our
# case-study, it makes sense to use the pre-specified links
# of PWD parcels to building footprints. It would be valuable
# to benchmark different spatial joining approaches of 
# PWD parcels and footprints to guide best practices for
# these kinds of joins for data that doesn't have pre specified links
# This will be the most common use case since building footprint data
# is rarely made available from the municipality and often comes
# from an external source

# Link properties to spatial data

In [5]:
# NSI and appraiser datasets have building footprint spatial references
# We will use these, and their unique ids, to link the structure
# inventories to spatial data like flood zones, depth grids, 
# reference data, and tabular data (which often can be made through shared
# references like zip codes or census tracts)

## Hazard data

## Reference data 

In [13]:
# Drop the following columns
drop_cols = ['type', 'geometry.type', 'geometry.coordinates', 'index_right']
nsi_gdf = nsi_gdf.drop(columns=drop_cols)

# Remove "properties" from columns
col_updates = [x.replace("properties.", "") for x in nsi_gdf.columns]
nsi_gdf.columns = col_updates

In [17]:
# Write the NSI data to interim
nsi_gdf.to_file(join(EXP_DIR_I, 'nsi.gpkg'), driver='GPKG')

## Depth Grids

# Prepare depth-damage functions